# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 20% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [523]:
%reset -f
import pandas as pd
import numpy as np
! pip install regex
import regex as re


# Filter all warnings. If you would like to see the warnings, please comment the two lines below.
# import warnings
# warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [524]:
##########################################################################################################
# ENERGY DATA
to_replace = ["Republic of Korea",
              "United States of America",
              "United Kingdom of Great Britain and Northern Ireland",
              "China, Hong Kong Special Administrative Region"]

replacement = ["South Korea",
               "United States",
               "United Kingdom",
               "Hong Kong"]

Energy = pd.read_excel('assets/Energy Indicators.xls')   # Load data from the Excel file
Energy.name='Energy'                                     # Name the DataFrame

Energy = (Energy[17:244]   # Exclused header and footer
        .drop(['Unnamed: 0','Unnamed: 1'],axis=1)        # Drop the first two columns
        .reset_index(drop=True)                          # Reset index and drop existing ones
        .replace('...',np.NaN)                           # Replace missing values with NaN
        .rename(columns={'Unnamed: 2':'Country',         # Rename columns
                         'Unnamed: 3':'Energy Supply',
                         'Unnamed: 4':'Energy Supply per Capita',
                         'Unnamed: 5':'% Renewable'}))

# Convert Energy to GigaJoules
Energy['Energy Supply'] = Energy['Energy Supply']*10**6  

# Remove numbers from Country names and strip any white spaces
Energy['Country'] = Energy['Country'].apply(lambda name: re.split(r'[0-9]+', name)[0].strip())

# Remove brackets from names and strip any white spaces
Energy['Country'] = Energy['Country'].apply(lambda name: re.split(r'[(]', name)[0].strip())

# Rename countries in place
Energy.replace(to_replace,replacement,inplace=True)  

# Set country as index
Energy=Energy.set_index('Country')                           

##########################################################################################################

# GDP DATA

to_replace = ["Korea, Rep.",
              "Iran, Islamic Rep.",
              "Hong Kong SAR, China"
             ]

replacement = ["South Korea",
               "Iran",
               "Hong Kong"
              ]

# Load GDP Data
GDP = pd.read_csv('assets/world_bank.csv',sep=',')

# Name the dataframe
GDP.name = 'GDP'


# Set the third row as the column names
GDP.columns = GDP.iloc[3].values

# Convert the floating numbers to string without Decimal points
GDP.columns.values[4:]= [str(int(x)) for x in GDP.columns.values[4:]]


# Remove the name of the columns
GDP.columns.name=None


GDP=  (GDP[4:]                                        # Start DataFrame from the 4th index (row) to the end
      .reset_index(drop=True)                         # Reset the Indices
      .replace(to_replace,replacement)                # Rename the countries
      .rename(columns={'Country Name':'Country'}))    # Rename Country column
        

# Set country as index
GDP=GDP.set_index('Country')                           

##########################################################################################################
# ScimEn DATA
ScimEn = pd.read_excel('assets/scimagojr-3.xlsx')

# Name the dataframe
ScimEn.name = 'ScimEn'

# Set Country as index
ScimEn = ScimEn.set_index('Country')
##########################################################################################################

merged = pd.concat([Energy, GDP, ScimEn], axis=1, join="outer")

# For rearranging columns
cols = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 
           'Energy Supply', 'Energy Supply per Capita', '% Renewable', 
           '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']

# Rearrange the columns
merged = merged[cols]
merged.sort_values(by=['Rank'],inplace=True)

##########################################################################################################

def answer_one():
    return merged[:15]

In [525]:
assert type(answer_one()) == pd.DataFrame, "Q1: You should return a DataFrame!"

assert answer_one().shape == (15,20), "Q1: Your DataFrame should have 20 columns and 15 entries!"


In [526]:
# Cell for autograder.


### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [527]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [528]:
def answer_two():
    inner_merge = pd.concat([Energy, GDP, ScimEn], axis=1, join="inner")
    return len(merged) - len(inner_merge)





In [529]:
assert type(answer_two()) == int, "Q2: You should return an int number!"


### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [539]:
def answer_three():
    GDP['avgGDP']=GDP.loc[:,'2006':'2015'].apply(lambda x: np.nanmean(x),axis=1)
    return GDP.loc[list(answer_one().index)].sort_values(by='avgGDP',ascending=False)['avgGDP']


pandas.core.series.Series

In [446]:
assert type(answer_three()) == pd.Series, "Q3: You should return a Series!"


### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [447]:
def answer_four():
    # YOUR CODE HERE
    
    fifthLargestGDP=merged.sort_values(by='avgGDP',ascending=False).iloc[5]
    return fifthLargestGDP['2015']-fifthLargestGDP['2006']
    

answer_four()

1787625194095.8008

In [448]:
# Cell for autograder.


### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

In [449]:
def answer_five():
    # YOUR CODE HERE
    return np.nanmean(merged['Energy Supply per Capita'])

answer_five()

90.66666666666667

In [450]:
# Cell for autograder.


### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [451]:
def answer_six():
    # YOUR CODE HERE
    return (merged['% Renewable'].idxmax(),merged.loc[merged['% Renewable'].idxmax(),'% Renewable'])

answer_six()

('Albania', 100.0)

In [452]:
assert type(answer_six()) == tuple, "Q6: You should return a tuple!"

assert type(answer_six()[0]) == str, "Q6: The first element in your result should be the name of the country!"


### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [459]:
def answer_seven():
    # YOUR CODE HERE
    # SC: Self-Citations
    # TC: Total Citations
    merged['SC/TC']=merged['Self-citations']/merged['Citations']
    
    return (merged['SC/TC'].idxmax(),merged.loc[merged['SC/TC'].idxmax(),'SC/TC'])
 
answer_seven()

('French Guiana', 1.0)

In [460]:
assert type(answer_seven()) == tuple, "Q7: You should return a tuple!"

assert type(answer_seven()[0]) == str, "Q7: The first element in your result should be the name of the country!"


### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

In [455]:
def answer_eight():
    # YOUR CODE HERE
    merged['Population']=merged['Energy Supply']/merged['Energy Supply per Capita']
    thirdMostPopulous = merged.sort_values(by='Population',ascending=False).iloc[2]
    return thirdMostPopulous.name
 
answer_eight()

'United States'

In [456]:
assert type(answer_eight()) == str, "Q8: You should return the name of the country!"


### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [458]:
def answer_nine():
    import scipy.stats as stats
    # YOUR CODE HERE
    merged['Citable Documents per Person'] =merged['Citable documents']/merged['Population']
    
    df=merged[['Citable Documents per Person','Energy Supply per Capita']].dropna()
    
    corr, pval=stats.pearsonr(df['Citable Documents per Person'],df['Energy Supply per Capita'])
    
    return corr

answer_nine()

ValueError: array must not contain infs or NaNs

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
plot9()

In [ ]:
assert answer_nine() >= -1. and answer_nine() <= 1., "Q9: A valid correlation should between -1 to 1!"


### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    # YOUR CODE HERE
    merged['Above Median'] = 0
    merged.loc[merged['% Renewable']>=np.nanmedian(merged['% Renewable']),'Above Median']=1
    return merged[merged['Above Median']==1]['% Renewable']

answer_ten()

In [ ]:
assert type(answer_ten()) == pd.Series, "Q10: You should return a Series!"


### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}


def sum_mean_std(group):    
    # now broadcast our formula and create a new column
    group["size"]=len(group)
    group["sum"]=np.nansum(group["Population"])
    group['mean']=np.nanmean(group["Population"])
    group['std']=np.nanstd(group["Population"])
    return group

    

    
    
def answer_eleven():
    # Get the population column from the dataframe
    df=pd.DataFrame(merged.loc[list(ContinentDict.keys())]['Population']).reset_index()

    # Add continent column to the dataframe
    df['Continent']=list(ContinentDict.values())

    # Rearrange the columns of the dataframe
    df = df[['Continent','Country','Population']]

    # Now just apply this to the groups
    df=df.groupby(['Continent']).apply(sum_mean_std)

    df =(df.drop(['Population','Country'],axis=1)
              .set_index(['Continent']))
    
    return df.groupby('Continent').agg(np.nanmean)
    
answer_eleven()





In [ ]:
assert type(answer_eleven()) == pd.DataFrame, "Q11: You should return a DataFrame!"

assert answer_eleven().shape[0] == 5, "Q11: Wrong row numbers!"

assert answer_eleven().shape[1] == 4, "Q11: Wrong column numbers!"


### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    # YOUR CODE HERE

    Top15 = answer_one().reset_index()

    Top15['Category']=pd.cut(Top15['% Renewable'],5)

    Top15['Continent']=list(ContinentDict.values())

    Top15 = Top15[['Continent','Category','Country']]


    Top15.set_index(['Continent','Category'],inplace=True,drop=True)

    Top15.sort_index(inplace=True)

    Top15=Top15.groupby(level=(0,1)).agg(', '.join).dropna()

    series = pd.Series(Top15['Country'],index=Top15.index)

    return series

answer_twelve()

In [ ]:
assert type(answer_twelve()) == pd.Series, "Q12: You should return a Series!"

assert len(answer_twelve()) == 9, "Q12: Wrong result numbers!"


### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*

In [ ]:
def answer_thirteen():
    # YOUR CODE HERE
    Top15 = answer_one()
    Top15['Population'] = merged['Population'].apply('{0:,.2f}'.format)
    return Top15['Population']


In [ ]:
assert type(answer_thirteen()) == pd.Series, "Q13: You should return a Series!"

assert len(answer_thirteen()) == 15, "Q13: Wrong result numbers!"


### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
plot_optional()